# Technical stuff and disclaimer:
Disclaimer: Several images are left out from the online material that have only been used in a seminar talk. The material can be found in the book:
* Karniadakis, G., & Sherwin, S. (2005-06-02). Spectral/hp Element Methods for Computational Fluid Dynamics. Oxford University Press, see https://oxford.universitypressscholarship.com/view/10.1093/acprof:oso/9780198528692.001.0001/acprof-9780198528692. 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#!pip install jupyterthemes
#!pip install --upgrade jupyterthemes!
#!jt -t monokai -T -N

In [2]:
from ipywidgets import interact, interactive_output, interact_manual, interactive, FloatSlider, IntSlider, Dropdown
from IPython.display import display
from math import sin, cos, exp, log, sqrt
from ngsolve import * 
from mesh1d import *
from draw1d import * 
from specials import *

In [3]:
#!jt -r

# Higher Order FEM in 1D

* Method of Weighted Residuals (what is the role of finite element **spaces**)
* 1D piecewise linear trial/expansion bases (the simplest example)
* 1D finite elements:  Global vs. local expansions (the basis matters!)
* Construction of higher order finite elements 
* Quadrature 
* (Error bounds)

## Method of Weighted Residuals

Consider a linear differential equation on a domain $\Omega$ by
$$
L(u) = f
$$

### Approximate solution
$$
u_h(x,t) = u_0(x,t) + \sum_{i=1}^{N_{dof}} \vec{u}_i(t) \varphi_i(x)
$$
where $\varphi_i(x)$ are the *trial* (or expansion) functions. 

#### Desired equation
$$
    R(u_h) = L(u_h) - f \equiv 0
$$
where *R* is the *residual*.

##### Problem: $u_h$ is $N_{dof}$-dimensional, but the equation is posed in infinite dimensions.

#### Test / weighting functions:
* Introduce a restriction of the residual $R$
* Only ask for 
$$
    \langle \psi_i(x),R(u_h) \rangle = 0, \quad \text{for } i=1,..,N_{dof}
$$
for *test* (or *weighting*) functions $\psi_i$ where $\langle \cdot, \cdot \rangle$ is a general duality pairing (think of the scalar product $\int_{\Omega} \cdots~ dx$ for now)


* Different types of test functions lead to different schemes

<center><img src="weightfcts.png"/></center>

### Galerkin formulation

Find $u_h \in V_h$ (with $V_h = \operatorname{span} \{ \varphi_i \}$) so that
$$
    \langle v_h, R(u_h) \rangle = 0 \quad \forall v_h \in V_h
$$
or equivalently
$$
    \langle \varphi_i, R(u_h) \rangle = 0 \quad \text{for } i = 1,..,N_{dof}.
$$



Now: $N_{dof}$ unknowns and $N_{dof}$ equations.

#### Weak formulation
$R(u_h)$ does not (even) need to be a $C^0$ or $L^2$ function, i.e. $\langle \cdot, \cdot \rangle$ could also be a duality pairing with lower regularity requirements on the second argument. 

For example: 

For the Poisson problem $- \Delta u = f$ in $\Omega $ with $u = 0$ on $\partial \Omega$ we know that: 

Find $u \in H^1_0(\Omega)$, s.t. $(\nabla u, \nabla v)_{\Omega} = (f,v)_{\Omega}$ for all $v \in H^1_0(\Omega)$ 

is a proper *weak formulation*.

The standard Galerkin formulation of the Poisson problem is then obtained with 
$$
R: H^1_0 \to H^{-1} := (H^1_0)^*, \qquad \langle v , R(u) \rangle_{H^1_0 \times H^{-1}} = (\nabla v, \nabla u) - ( v, f)
$$

**The choice of the trial / expansion function spaces matters!**

## 1D piecewise linear trial/expansion bases

Recall: Standard linear finite element

In [4]:
align(interactive(DrawBasisFunction, N=[4,6,8], i=IntSlider(min=0, max=41, step=1, continuous_update=False),
                  order=[1,2,3]))

interactive(children=(Dropdown(description='N', options=(4, 6, 8), value=4), IntSlider(value=0, continuous_upd…

### Affine mappings

 Reference domain:
 $$
 \hat{T} = (-1,1) \qquad (\text{often also } \hat{T} = (0,1) )
 $$

Every *physical* element (interval) $(a,b)$ can be mapped to $\hat{T}$ by an affine mapping:
$$
\Phi_i: \hat{T} \to T_i = (x_{i-1},x_i)
$$

Global modes $\varphi_i$ can be represented in terms of local elemental expansion modes $\phi_i$:

$$
\varphi_i = \left\{ \begin{array}{cl} \frac{x-x_{i-1}}{x_i-x_{i-1}} & x \in T_i \\ \frac{x-x_{i+1}}{x_i-x_{i+1}} & x \in T_{i+1} \\ 0 & \text{otherwise} \end{array} \right.
= \left\{ \begin{array}{cl} \phi_1( \Phi_i^{-1}(x) )  & x \in T_i \\ \phi_0( \Phi_{i+1}^{-1}(x) ) & x \in T_{i+1} \\ 0 & \text{otherwise} \end{array} \right.
$$


In [5]:
align(interactive(DrawBasisFunction, N=[1], i=IntSlider(min=0, max=1, step=1, continuous_update=False),
                  order=[1]))

interactive(children=(Dropdown(description='N', options=(1,), value=1), IntSlider(value=0, continuous_update=F…

### Parametric mappings
The maping $\Phi_i$ from *local* coordinates (in $\hat{T}$) to *global* coordinates (in $T_i$) can be seen as expanding the global coordinate as a finite element expansion:
$$
  x = \Phi_i(\hat{x}) = \phi_0(\hat{x}) x_{i-1} + \phi_1(\hat{x}) x_i, \hat{x} \in \hat{T}
$$
Hence, $x$ is represented by a finite element function. If the polynomial degree of this mapping $\Phi_i$ and the finite element trial/test space are the same, the method is denoted as *iso-parametric*, otherwise *sub*- or *super*-parametric.

## 1D finite elements:  Global vs. local expansions

The global finite element space

The space of continuous, piecewise polynomials:
$$
V_h = \{ u_h \mid u_h \in H^1, u_h(\Phi_i(\hat{x})) \in \mathcal{P}^k(\hat{T}), i=1,..,N_{el}\}
$$
Note: $u_h$ is a piecewise polynomial **after** mapping with $\Phi_i$, i.e. w.r.t. to $\hat{T}$. 

#### Global and local expansion
For $u_h \in V_h$ we have the representations:
$$
u_h(x) = \sum_{i=1}^{N_{dof}} \varphi_i(x) \vec{u}_i = \sum_{i=1}^{N_{el}} \sum_{l=0}^k \phi_l(\underbrace{\Phi_i^{-1}(x)}_{\hat{x}}) \vec{u}_l^i
$$
with:
1. the global representation ($\vec{u}_i$, $i=1,..,N_{dof}$) and
2. an element local representation ($\vec{u}_l^i$, $i=1,..,N_{el}$, $l=0,..,k$) 

why $N_{dof} \not = N_{el} \cdot (k+1) $?

* Continuity constraint!

Example:

For $k=1$ we have
$u_1^i = u_0^{i+1}$.

Local expansion basis is used for local computations:
* element matrices
* element vectors
* integrals

Global expansion basis for
* setup of global matrix
* setup of global vectors

#### Transformation global to local. Example: $N_{el}=3$, $k=1$
$$
\vec{u}_{loc} = 
(\vec{u}_l^i)_{i,l} =  
\left( \begin{array}{c} 
u_0^1 \\ 
u_1^1 \\ 
u_0^2 \\ 
u_1^2 \\ 
u_0^3 \\ 
u_1^3 
\end{array}  \right)
=
\underbrace{
\left( \begin{array}{cccc} 
1 &   &  & \\ 
  & 1 &  & \\ 
  & 1 &  & \\ 
  &   & 1& \\ 
  &   & 1& \\ 
 &   &  &1 
\end{array}  \right)
}_{E}\cdot 
\underbrace{
\left( \begin{array}{c} 
u_1 \\ 
u_2 \\ 
u_3 \\ 
u_4 \\ 
\end{array}  \right)
}_{u_{glob}}
$$

* $E$ *scatters* information from global to all local elemens
* $E^T$ *gathers*/*assembles* (sums) local contributions to global ones

**Implementational note:**
$E$ and $E^T$ are typically not implemented as matrices but through a simple list per element:

In [ ]:
mesh = Mesh1D(10)
Vh = H1(mesh,order=1)
for el in Vh.Elements():
    for loc, glob in enumerate(el.dofs):
        print(f"loc.dof {loc:3} -> glob.dof {glob:3}")

Assembling of matrices / vectors:

$$
f_{i} = f(\varphi_i) = \sum_n f_{T_n}(\varphi_i), \qquad
\vec{f}_{glob} = E^T \cdot \vec{f}_{loc}
$$


$$
A_{ij} = a(\varphi_i,\varphi_j) = \sum_n a_{T_n}(\varphi_i,\varphi_j), \qquad
\vec{A}_{glob} = E^T \cdot \vec{A}_{loc} \cdot E
$$

**Illustration for $\vec{A}_{loc}$:**

In [ ]:
align(interactive(SpyDG,N=[1,2,3,4,8,16],order=[1,2,3,4]))

This matrix will typically not be computed!

In [ ]:
align(interactive(Spy,N=[1,2,3,4,8,16],order=[1,2,3,4]))


## Construction of higher order expansions

### Modal vs. nodal expansions. Simple examples:

1. Moment expansion:
$$
\phi_l^A(\hat{x}) = \hat{x}^l, \qquad l= 0,..,k
$$
2. Lagrange expansion:
$$
\phi_l^B(\hat{x}) = \frac{\prod_{n=0,n\not = k}^k (x - x_n)}{\prod_{n=0,n\not = k}^k (x_l - x_n)} , \qquad l= 0,..,k
$$
3. Legendre expansion:
$$
\phi_l^B(\hat{x}) = L_l(\hat{x}), \qquad l= 0,..,k
$$
with the $L^2((-1,1))$-orthogonal Legendre polynomials, i.e.
$$
\int_{-1}^1 L_m(\hat{x}) L_n(\hat{x}) ~ d\hat{x} = \left( \frac{2}{2p+1} \right) \delta_{mn}
$$

### Properties of expansions:
##### Hierarchical: 
$$
\{ \phi_l \}_{l=0,..,k}  \subset \{ \phi_l \}_{l=0,..,k+1} ?
$$

|Moment expansion|Lagrange expansion|Legendre expansion|
|-|-|-|
|X|-|X|

##### Suitability for efficient assembling and solution of linear systems:

Example: $(v,u)_{\Omega} = (f,v)_{\Omega}$

Requires 
* the computation  of the local mass matrices
$$
  (M_{loc}^{el})_{ij} = (\phi_i,\phi_j)_{\hat{T}}
$$
* the assembly to a global mass matrix
* the solution of a linear system

###### Comparison of different bases and condition number of element matrix:
$$
\kappa_2 = \Vert M_{loc}^{el} \Vert_2 \Vert (M_{loc}^{el})^{-1} \Vert_2  
$$

![cond](cond.png)

#### Global continuity
Constraint needed to impose continuity:

|expansion|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;constraint|
|-|-|
|Moment expansion| $\sum_l \vec{u}_l^i = \sum_l (-1)^l \vec{u}_l^{i+1}$|
|Lagrange expansion| $\vec{u}_k^i = \vec{u}_0^{i+1}$|
|Legendre expansion| $\sum_l \vec{u}_l^i = \sum_l (-1)^l \vec{u}_l^{i+1}$|


**Too involved! Reduce modes/shape functions that have nonzero boundary values!**

We want a basis so that $\phi_l(-1) = \left\{ \begin{array}{cc} 1 & l = 0 \\0 & \text{ otherwise} \end{array}\right.$ and $\phi_l(1) = \left\{ \begin{array}{cc} 1 & l = k \\0 & \text{ otherwise} \end{array}\right.$.

This is the decomposition into *boundary* and *interior*. 

The continuity constraint requires only
$\vec{u}_k^i = \vec{u}_0^{i+1}$ (as in the Lagrange case). 

Best of both worlds: Basis with
* decomposition into boundary and interior functions
* well-conditioning, i.e. slow grow of condition number with $k$.

#### Jacobi polynomials:

The Jacobi polynomials are orthogonal polynomials as the Legendre polynomials, but w.r.t. a special inner product:
$$
 (u,v)_{\alpha,\beta} := \int_{-1}^1 (1-\hat{x})^\alpha (1+\hat{x})^{\beta} u v d\hat{x}
$$

The Jacobi polynomials $P_l^{\alpha,\beta}(\hat{x})$ have 

$$
 (P_m^{\alpha,\beta},P_n^{\alpha,\beta})_{\alpha,\beta} := \int_{-1}^1 (1-\hat{x})^\alpha (1+\hat{x})^{\beta} P_m^{\alpha,\beta}(\hat{x}) P_nm^{\alpha,\beta}(\hat{x}) d\hat{x} = C(\alpha,\beta,m) \delta_{mn}.
$$

(special cases: $\alpha=\beta=0$: Legendre, $\alpha=\beta=-\frac12$: Chebychev)

#### $k$th order modal finite element

$$
\phi_l(\hat{x}) = 
\left\{ 
\begin{array}{cl} 
\frac{1-\hat{x}}{2}, & l=0, \\
\frac{1-\hat{x}^2}{4} \cdot P_{l-1}^{1,1}(\hat{x}), & 0 < l < k, \\
\frac{1+\hat{x}}{2}, & l=k.
\end{array}
\right.
$$



![modal](modal.png)

Remark: $P_{l-1}^{1,1}(\hat{x})$ can also be characterized through integrated Legendre polynomials:
$$
\frac{1-\hat{x}}{2} \frac{1+\hat{x}}{2} P_{l-1}^{1,1}(\hat{x}) = -2 k \int_{-1}^{\hat{x}} L_l(s) ds
$$

**Variants for interior shapes**
* as before: $\frac{1-\hat{x}}{2} \frac{1+\hat{x}}{2} P_{l-1}^{1,1}(\hat{x})$
* Legendre-based: $\frac{1-\hat{x}}{2} \frac{1+\hat{x}}{2} L_{l-1}(\hat{x})$
* higher $\alpha/\beta$: $\frac{1-\hat{x}}{2} \frac{1+\hat{x}}{2} P_{l-1}^{2,2}(\hat{x})$



**Sparsity element mass matrix:**
![sparseM](sparseM.png)

**Sparsity element stiffness matrix:**
![sparseA](sparseA.png)

#### "Spectral elements":
* Lagrange elements are not suitable for higher **if equidistant nodes are used**
* Choosing the zeros of Gauss-Lobatto polynomials (or Chebychev polynomials) can solve the conditioning issues

![spectral](spectral.png)


#### sparsity

element matrices

|modal| spectral|
|-|-|
| sparse | full|

**Trick:** Use the nodes of the Lagrange polynomials for quadrature (zeros of orthogonal polynomials):

**lumped** element matrices

|modal| spectral|
|-|-|
| sparse | sparse|

## Quadrature

Find nodes $\hat{x}_i$ and weights $\omega_i$ (independent of $f$) so that 
$$
I(f) = \int_{-1}^1 f(\hat{x}) d\hat{x} \approx Q(f) = \sum_{i=1}^{q} \omega_i u(\hat{x}_i)
$$
holds as good as possible (for smooth $f$).

### Quadrature weights
Assume the quadrature points are fixed. 

Then, we can determine the weights so that they maximize the exactness degree of a quadrature rule (order for which integration is still exact):

Asking for $I(f) = Q(f)$ for all polynomials up to degree $q-1$ especially requires
$I(h_i) = Q(h_i)$ for
$$
h_i(\hat{x}) = \frac{\prod_{j=1,j\not = i}^q (x-x_j)}{\prod_{j=1,j\not = i}^q (x_i-x_j)} 
$$
the Lagrange polynomials to the nodes $\hat{x}_i$ of the quadrature rule.

One easily sees:
$$\omega_i = Q(h_i) = \int_{-1}^1 h_i(\hat{x}) d \hat{x}$$ 

### Gaussian(-Legendre) quadrature:
For Gaussian quadrature the nodes are chosen carefully, as the zeros of an orthogonal polynomial. let $\ell_{q}$ be the orthogonal polynomial of order $q$ then, we can decompose any $f \in \mathcal{P}^{2q-1}$ as (polynomial division)
$$
  f = p ~ \ell_q + r, \qquad \text{with } p,r \in \mathcal{P}^{q-1}
$$

* Now ask for $I(p ~ \ell_q) = Q(p ~ \ell_q)$ and $I(r) = Q(r)$ separately.

* For the first condition we note that $I(p ~ \ell_q)$ due to the orthogonality condition. 

* To obtain the same for the quadrature we take the nodes $\hat{x}_i$ as the zeros of $\ell_q$ so that $Q(p ~ \ell_q)=0$

* Having fixed the nodes, we determine the weights so that $I(r) = Q(r)$ for any $r \in \mathcal{P}^{q-1}$ (see last slide)

**Remark**: The previous construction is independent of the inner product under consideration.

### Important variants: Gauss-Radau-Legendre and Gauss-Lobatto-Legendre

* It is often desirable to include one or both end points of an interval. 
* Then, the orthogonal polynomial $\ell_{q-1}$ or $\ell_{q-2}$ is taken as a basis. 
* Its zeros together with one/both end point(s) determine the nodes.
* The weights are determined as before. 

**Remark**: 
 * As the quadrature accuracy of Gauss-(xxx)-Legendre rules are sufficiently accurate in the finite element context, one often replaces exact (or more exact) quadrature rules with *compatible* quadrature rules. 
 * If finite element basis and quadrature rule math, the mass matrix becomes **diagonal**.

## Convergence of h- vs. p-type FEM
Poisson problem, with equispaced elements of size $h$ and polynomial degree $k$:
$$
 \Vert u - u_h \Vert_{H^1} \leq C h^{\mu-1} k^{1-\eta} \Vert u \Vert_{H^{\eta}}
$$
with $\mu = \min(\eta,k+1)$.

If $u$ is smooth, i.e. we can choose $\eta \geq k+1$ and have 
$$
 \Vert u - u_h \Vert_{H^1} \leq C \left(\frac{h}{k}\right)^{k} \Vert u \Vert_{H^{k+1}}.
$$

which for $h=const$, $k \to \infty$ is *exponential convergence*.